# Lab 8-3: MNIST backpropagation(High-Level) - MNIST 데이터셋 분류하기
모두를 위한 딥러닝 시즌2의 [`lab-08_4_mnist_back_prop.ipynb`](https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-08_4_mnist_back_prop.ipynb)에 해당하는 내용이다. 모두딥 시즌2에서는 직접 미분을 하는 과정까지 다루었으나, 여기서는 high level에서 구현하고, 나머지는 따로 다룬다.

In [1]:
#Import
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

#parameters
learning_rate = 0.5
batch_size = 10

#load data (MNIST dataset)
mnist_train = dsets.MNIST(root='MNIST_data/',
                    train=True,
                    transform=transforms.ToTensor(),
                    download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                    train=False,
                    transform=transforms.ToTensor(),
                    download=True)

#dataset loader
data_loader = torch.utils.data.DataLoader(dataset= mnist_train,
                                          batch_size= batch_size,
                                          shuffle= True,
                                          drop_last= True)

In [2]:
#model (high level)
linear1 = torch.nn.Linear(28 * 28, 30, bias= True).to(device)
linear2 = torch.nn.Linear(30, 10, bias= True).to(device)
sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

criterion =  torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

In [3]:
X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)[:1000]
Y_test = mnist_test.targets.to(device)[:1000]

#train & test
i = 0
while i < 10001:
  for X, Y in data_loader:
    i += 1
    X = X.view(-1, 28*28).to(device)
    Y=Y.to(device)

    hypothesis = model(X)
    cost = criterion(hypothesis, Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if i % 1000 == 0:
      with torch.no_grad():
        hypothesis = model(X_test)
        accurate_prediction = (hypothesis.argmax(1) == Y_test).sum()
        print(accurate_prediction.item())

823
849
866
883
897
884
890
895
908
908
903
900
